# **Default Setting**

* Python: 3.6.9
* ML Framework: tf-nightly-gpu 2.5.0-dev20201208
* CPU: AMD Ryzen 5 5600X 6-Core Processor
* GPU: GeForce RTX 3070 (8G) (CUDA 11.1)
* RAM: 32G
* Platform: linux 20.04 LTS

In [1]:
%env NOTEBOOKNAME try7
%env LOCAL_DATA_PATH data
%env SUBMISSION_PATH submission

env: NOTEBOOKNAME=try7
env: LOCAL_DATA_PATH=data
env: SUBMISSION_PATH=submission


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa

import datetime
import glob
import os
import platform

import numpy as np
import pandas as pd

from collections import OrderedDict
from sklearn.model_selection import train_test_split

print(f"tf.__version__: {tf.__version__}")
print(f"tfa.__version__: {tfa.__version__}")

tf.__version__: 2.5.0-dev20201208
tfa.__version__: 0.11.2


/home/myunggyo/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:44: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201208). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# !python -V

In [5]:
# !cat /proc/cpuinfo

In [6]:
!nvidia-smi

Sat Dec 12 16:25:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     On   | 00000000:0A:00.0  On |                  N/A |
|  0%   40C    P8    25W / 220W |    365MiB /  7979MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# !df -h

In [8]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           31Gi       3.3Gi        24Gi       428Mi       3.4Gi        27Gi
Swap:         2.0Gi          0B       2.0Gi


In [9]:
# platform.platform()

In [10]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

# **Define Arguments**

In [11]:
args = OrderedDict({
    # Basic
    "SEED": 42,
    "TEST_SIZE": 0.2,
    "EPOCH": 10,
    "INIT_LR": 1e-3,
    
    # Dataframe
    "NUM_TEST_CSV": 81,
    
    # Dataset Pipeline
    "WINDOW_INP": 7 * 24 * 2, # 336, 7 days
    "WINDOW_TAR": 2 * 24 * 2, # 96, 2 days
    "WINDOW_SHIFT": 1 * 24 * 2, # 1 day
    "WINDOW_STRIDE": 1,
    "WINDOW_DR": True, # Drop remainders
    "AUTO": tf.data.experimental.AUTOTUNE,
    
    "GLOBAL_BATCH_SIZE": 128,
    
    # Model
    "NUM_UNITS": 128, # lstm units
    "NUM_FEATURES": 6, # dense featues
    "NUM_QUANTILE": 9,
    
})

args

OrderedDict([('SEED', 42),
             ('TEST_SIZE', 0.2),
             ('EPOCH', 10),
             ('INIT_LR', 0.001),
             ('NUM_TEST_CSV', 81),
             ('WINDOW_INP', 336),
             ('WINDOW_TAR', 96),
             ('WINDOW_SHIFT', 48),
             ('WINDOW_STRIDE', 1),
             ('WINDOW_DR', True),
             ('AUTO', -1),
             ('GLOBAL_BATCH_SIZE', 128),
             ('NUM_UNITS', 128),
             ('NUM_FEATURES', 6),
             ('NUM_QUANTILE', 9)])

# **Load Datasets**

## **Train / Validation**

In [23]:
df = pd.read_csv(os.path.join(os.environ["LOCAL_DATA_PATH"], "train", "train.csv"))
df.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


In [24]:
df.describe()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
count,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000
mean,547.000000,11.500000,15.000000,64.344121,234.792371,2.456033,56.793102,9.279928,17.790630
std,316.102148,6.922252,15.000143,103.897125,349.684583,1.426874,22.052875,10.179741,25.759955
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.590000,-19.000000,0.000000
25%,273.000000,5.750000,0.000000,0.000000,0.000000,1.400000,39.697500,1.000000,0.000000
50%,547.000000,11.500000,15.000000,0.000000,0.000000,2.200000,57.600000,9.000000,0.000000
75%,821.000000,17.250000,30.000000,87.000000,469.000000,3.200000,72.770000,17.000000,32.089890
max,1094.000000,23.000000,30.000000,528.000000,1059.000000,12.000000,100.000000,35.000000,99.913939


In [25]:
tr_df, vl_df = train_test_split(
    df, 
    test_size = args["TEST_SIZE"], 
    random_state = args["SEED"])

tr_df.shape, vl_df.shape

((42048, 9), (10512, 9))

In [26]:
def preprocessing(df, is_training = True):
    # Drop timestampes.
    try:
        df = df.drop(["Day", "Hour", "Minute"], axis = 1)
    except:
        pass
    
#     # Normalize.
#     for column in df.columns[:-1]:
#         if is_training:
#             args[f"{column}_MAX"] = df[column].max()
#             args[f"{column}_MIN"] = df[column].min()
        
#         df[column] = (df[column] - args[f"{column}_MIN"]) / (args[f"{column}_MAX"] - args[f"{column}_MIN"])
        
    return df

In [27]:
tr_df = preprocessing(tr_df)
vl_df = preprocessing(vl_df, is_training = False)

In [28]:
tr_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,42048.000000,42048.000000,42048.000000,42048.000000,42048.000000,42048.000000
mean,63.969654,234.699439,2.453686,56.808972,9.253377,17.719547
std,103.316355,349.813779,1.423941,22.072544,10.179447,25.678521
min,0.000000,0.000000,0.000000,7.590000,-19.000000,0.000000
25%,0.000000,0.000000,1.400000,39.660000,1.000000,0.000000
50%,0.000000,0.000000,2.200000,57.620000,9.000000,0.000000
75%,87.000000,470.000000,3.200000,72.860000,17.000000,31.997818
max,528.000000,1059.000000,12.000000,100.000000,35.000000,99.747138


In [29]:
vl_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,10512.00000,10512.000000,10512.000000,10512.000000,10512.000000,10512.000000
mean,65.84199,235.164098,2.465420,56.729620,9.386130,18.074962
std,106.18024,349.183694,1.438577,21.974959,10.180708,26.082442
min,0.00000,0.000000,0.100000,8.000000,-17.000000,0.000000
25%,0.00000,0.000000,1.400000,39.770000,1.000000,0.000000
50%,0.00000,0.000000,2.200000,57.550000,9.000000,0.000000
75%,89.00000,464.000000,3.200000,72.300000,17.000000,32.370490
max,523.00000,1057.000000,11.800000,100.000000,35.000000,99.913939


## **Test**

In [30]:
ts_filenames = [os.path.join(os.environ["LOCAL_DATA_PATH"], "test", f"{i}.csv") for i in range(args["NUM_TEST_CSV"])]
ts_dfs = [preprocessing(pd.read_csv(ts_filename), is_training = False) for ts_filename in ts_filenames]
ts_df = pd.concat(ts_dfs)

In [31]:
ts_df.head()

,DHI,DNI,WS,RH,T,TARGET
0,0,0,2.7,34.42,0.0,0.0
1,0,0,2.7,34.17,0.1,0.0
2,0,0,2.7,34.23,0.2,0.0
3,0,0,2.7,33.99,0.3,0.0
4,0,0,2.8,33.97,0.4,0.0


In [32]:
ts_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,27216.000000,27216.000000,27216.000000,27216.000000,27216.000000,27216.000000
mean,59.072751,251.124045,2.272002,46.394399,9.746304,18.235096
std,91.605047,358.131478,1.271604,19.176784,11.286151,26.115609
min,0.000000,0.000000,0.100000,4.910000,-15.700000,0.000000
25%,0.000000,0.000000,1.300000,31.077500,0.400000,0.000000
50%,6.000000,0.000000,2.000000,46.830000,8.700000,0.563069
75%,83.000000,506.000000,2.900000,60.880000,18.300000,32.557286
max,507.000000,1079.000000,11.000000,92.570000,36.700000,99.450872


# **Make Dataset Pipelines**

In [33]:
# @tf.function
# def _flat_fn_tr(x):
#     return x.batch(args["WINDOW_INP"] + args["WINDOW_TAR"])


# @tf.function
# def _flat_fn_ts(x):
#     return x.batch(args["WINDOW_INP"])


# @tf.function
# def _split_window(features):
#     return tf.split(features, [args["WINDOW_INP"], args["WINDOW_TAR"]], axis = 0)

In [34]:
@tf.function
def _split_window(features):
    return tf.split(features, [5, 1], axis = 0)

In [40]:
tr_tensor = tf.constant(tr_df, dtype = tf.float32)
vl_tensor = tf.constant(vl_df, dtype = tf.float32)
ts_tensor = tf.constant(ts_df, dtype = tf.float32)

tr_dataset = tf.data.Dataset.from_tensor_slices(tr_tensor
#                     ).window(args["WINDOW_INP"] + args["WINDOW_TAR"], args["WINDOW_SHIFT"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_tr
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
                    ).cache(
                    ).prefetch(args["AUTO"])

vl_dataset = tf.data.Dataset.from_tensor_slices(vl_tensor
#                     ).window(args["WINDOW_INP"] + args["WINDOW_TAR"], args["WINDOW_SHIFT"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_tr
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
                    ).cache(
                    ).prefetch(args["AUTO"])

ts_dataset = tf.data.Dataset.from_tensor_slices(ts_tensor
#                     ).window(args["WINDOW_INP"], args["WINDOW_INP"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_ts
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
#                     ).cache(
                    ).prefetch(args["AUTO"])

# Print the shapes
print(f"tr_dataset: {tr_dataset.element_spec}")
print(f"vl_dataset: {vl_dataset.element_spec}")
print(f"ts_dataset: {ts_dataset.element_spec}")

tr_dataset: (TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))
vl_dataset: (TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))
ts_dataset: (TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))


## **Take Samples**

In [41]:
for element in tr_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 5) (128, 1)


In [42]:
for element in vl_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 5) (128, 1)


In [43]:
for element in ts_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 5) (128, 1)


# **Create Network**

In [53]:
class TARGET_MODEL(tf.keras.Model):
    def __init__(self, units, name):
        super(TARGET_MODEL, self).__init__(name = name)
        self.units = units
        
        self.dense1 = tf.keras.layers.Dense(units = self.units, activation = "relu")
        self.dense2 = tf.keras.layers.Dense(units = self.units, activation = "relu")
        self.dense3 = tf.keras.layers.Dense(units = 1)
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        
        return x        

# **Compile and Fit**

   * Repeat the last 24 hours.

In [61]:
model = TARGET_MODEL(
    units = 64,
    name = "model")

model.compile(
    loss = tf.keras.losses.MAE,
    optimizer = tf.keras.optimizers.Adam())

In [62]:
# TensorBoard callback.
log_dir = os.path.join(
    "logs", "fit", os.environ["NOTEBOOKNAME"], 
    datetime.datetime.now().strftime(f"{model.name}-%Y%m%d-%H%M%S"))
tb_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

_ = model.fit(
    tr_dataset,
    validation_data = vl_dataset,
    epochs = 500,
    verbose = 0,
    callbacks = [tb_callback])

In [63]:
%tensorboard --logdir logs/fit --host jupyter-server

Reusing TensorBoard on port 6006 (pid 3556), started -1 day, 16:09:06 ago. (Use '!kill 3556' to kill it.)

# **Predict**

In [64]:
model.evaluate(ts_dataset, verbose = 1)

213/213 [==============================] - 0s 802us/step - loss: 0.7587


0.7587046027183533

In [57]:
preds = [np.array(model.predict(ts_dataset))[..., -1] for model in models]
preds[0].shape

(81, 96)

In [58]:
submission = pd.read_csv(os.path.join(os.environ["LOCAL_DATA_PATH"], "sample_submission.csv"), index_col = "id")
submission.head()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
id,,,,,,,,,
0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
for column, pred in zip(submission.columns, preds):
    foo = np.reshape(pred, (-1,))
    foo = np.where(foo < 0, 0, foo) # clip by value
    
    submission[column] = foo

In [66]:
submission.head()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
id,,,,,,,,,
0.csv_Day7_0h00m,0.019628,0.009365,0.0,0.000000,0.112591,7.637287,14.906515,17.131758,17.608368
0.csv_Day7_0h30m,0.008961,0.009944,0.0,0.000000,0.124864,7.905676,15.397655,17.560627,17.959702
0.csv_Day7_1h00m,0.001013,0.008628,0.0,0.004815,0.133084,8.147173,15.504680,17.658821,18.036112
0.csv_Day7_1h30m,0.000000,0.007088,0.0,0.010220,0.139264,8.294870,15.531036,17.690212,18.056557
0.csv_Day7_2h00m,0.000000,0.005681,0.0,0.013636,0.143860,8.387443,15.534620,17.701130,18.060827


In [67]:
submission.to_csv(
    os.path.join(os.environ["SUBMISSION_PATH"], f"{os.environ['NOTEBOOKNAME']}_submission.csv"))